In [ ]:
pip install --quiet langchain langchain_community openai tiktoken langchain_openai

In [2]:
%pip install --upgrade --quiet  youtube-transcript-api

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=5nOX4x1cim0",
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30,
)
document = loader.load()

In [ ]:
import os
os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [ ]:
%pip install -Uqq langchain-weaviate

In [ ]:
%pip install --quiet weaviate-client

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore
weaviate_client = weaviate.connect_to_embedded()

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 24136


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
documents = text_splitter.split_documents(document)
db = WeaviateVectorStore.from_documents(document, embeddings, client=weaviate_client)

In [ ]:
template = """ Answer the question only based on the following context.
If you can't find answer in context, use your own knowlege
{context}

Question: {question}
"""
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template=template)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [ ]:
combine_docs_chain = create_stuff_documents_chain(llm=model, prompt=prompt)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
retrieval_chain.invoke({"input":"In which areas do students enrolled in the AI program at Durham College receive training?","question":"In which areas do students enrolled in the AI program at Durham College receive training?" })

{'input': 'In which areas do students enrolled in the AI program at Durham College receive training?',
 'question': 'In which areas do students enrolled in the AI program at Durham College receive training?',
 'context': [Document(metadata={'start_timestamp': '00:00:30', 'start_seconds': 30.0, 'source': 'https://www.youtube.com/watch?v=5nOX4x1cim0&t=30s'}, page_content='education to advance and establish your knowledge base and skill set to start a career in AI industry you will be given AI focused rigorous training in mathematics linear algebra statistics data science algorithms and Computing systems the program will teach the students the skills and tools they need to design and Implement an AI solution we have a comprehensive set of courses that not'),
  Document(metadata={'start_timestamp': '00:04:00', 'start_seconds': 240.0, 'source': 'https://www.youtube.com/watch?v=5nOX4x1cim0&t=240s'}, page_content='years AI is reshaping every aspect of our private and public life as we know it

In [ ]:
output = retrieval_chain.invoke({"input":"In which areas do students enrolled in the AI program at Durham College receive training?","question":"In which areas do students enrolled in the AI program at Durham College receive training?" })
print(output['answer'])

Students enrolled in the AI program at Durham College receive training in mathematics, linear algebra, statistics, data science, algorithms, and computing systems.
